# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat publications.tsv

pub_date	title	venue	category	excerpt	citation	url_slug	paper_url	slides_url
2024-02-17	Development of redox-type thermochemical energy storage module: A support-free porous foam made of CuMn2O4/CuMnO2 redox couple	Chemical Engineering Journal	manuscripts		"Your Name, You. (2009). ""Paper Title Number 1."" <i>Journal 1</i>. 1(1)."	paper-title-number-1	10.1016/j.cej.2024.149540	
2023-08-05	"An in-depth oxidation kinetic study of CuCrxMn1-xO2 (x = 0, 0.1, 0.3) for thermochemical energy storage at medium-high temperature"	Solar Energy Materials and Solar Cells	manuscripts		"Your Name, You. (2009). ""Paper Title Number 1."" <i>Journal 1</i>. 1(1)."	paper-title-number-2	10.1016/j.solmat.2023.112495	
2022-04-26	Effect of Cr Addition on Cu–Mn Spinel/Delafossite Redox Couples for Medium-High Temperature Thermochemical Energy Storage	ACS Applied Energy Materials	manuscripts		"Your Name, You. (2009). ""Paper Title Number 1."" <i>Journal 1</i>. 1(1)."	paper-title-number-3	10.1021/acsaem.2c00178	


## Import pandas

We are using the very handy pandas library for dataframes.

In [5]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [7]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,category,excerpt,citation,url_slug,paper_url,slides_url
0,2024-02-17,Development of redox-type thermochemical energ...,Chemical Engineering Journal,manuscripts,NaN,"Your Name, You. (2009). ""Paper Title Number 1....",paper-title-number-1,10.1016/j.cej.2024.149540,NaN
1,2023-08-05,An in-depth oxidation kinetic study of CuCrxMn...,Solar Energy Materials and Solar Cells,manuscripts,NaN,"Your Name, You. (2009). ""Paper Title Number 1....",paper-title-number-2,10.1016/j.solmat.2023.112495,NaN
2,2022-04-26,Effect of Cr Addition on Cu–Mn Spinel/Delafoss...,ACS Applied Energy Materials,manuscripts,NaN,"Your Name, You. (2009). ""Paper Title Number 1....",paper-title-number-3,10.1021/acsaem.2c00178,NaN
3,2021-07-08,Exploring Cu-Based Spinel/Delafossite Couples ...,ACS Applied Energy Materials,manuscripts,NaN,"Your Name, You. (2009). ""Paper Title Number 1....",Paper-title-number-4,10.1021/acsaem.1c01352,NaN
4,2021-04-05,Investigation of Sr-based perovskites for redo...,Journal of Energy Storage,manuscripts,NaN,"Your Name, You. (2009). ""Paper Title Number 1....",Paper-title-number-5,10.1016/j.est.2021.102501,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
import os

for row, item in publications.iterrows():
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]

    ## YAML variables
    md = "---\n"
    md += f'title: "{item.title}"\n'
    md += "collection: publications\n"

    category = item.get("category", "manuscripts")  # 取值或默认
    md += f"category: {category}\n"
    
    md += f"permalink: /publication/{html_filename}\n"

    #if len(str(item.excerpt)) > 5:
        #md += f"excerpt: '{html_escape(item.excerpt)}'\n"

    md += f"date: {item.pub_date}\n"
    md += f"venue: '{html_escape(item.venue)}'\n"

    if len(str(item.slides_url)) > 5:
        md += f"slidesurl: '{item.slides_url}'\n"

    if len(str(item.paper_url)) > 5:
        md += f"paperurl: '{item.paper_url}'\n"

    #md += f"citation: '{html_escape(item.citation)}'\n" #不需要citation
    md += "---\n"

    ## Markdown description for individual page
    if len(str(item.excerpt)) > 5:
        md += f"\n{html_escape(item.excerpt)}\n"

    #if len(str(item.slides_url)) > 5:
        #md += f"\n[Download slides here]({item.slides_url})\n"

    #if len(str(item.paper_url)) > 5:
        #md += f"\n[Download paper here]({item.paper_url})\n"

    #md += f"\nRecommended citation: {item.citation}" #不需要citation

    md_filename = os.path.basename(md_filename)

    with open(f"../_publications/{md_filename}", "w") as f:
        f.write(md)


These files are in the publications directory, one directory below where we're working from.

In [13]:
!ls ../_publications/

2021-04-05-Paper-title-number-5.md 2023-08-05-paper-title-number-2.md
2021-07-08-Paper-title-number-4.md 2024-02-17-paper-title-number-1.md
2022-04-26-paper-title-number-3.md


In [14]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
